In [8]:
from socceraction.spadl.statsbomb import convert_to_actions
from socceraction.data.statsbomb import StatsBombLoader
from socceraction.spadl.config import actiontypes, bodyparts
import socceraction.spadl as spadl
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, mean_squared_error
from xgboost import XGBClassifier, XGBRegressor
import math
import pickle
from mplsoccer import Pitch
import hashlib
import copy
import random

In [9]:
api = StatsBombLoader(root="data/statsbomb", getter="local")

In [10]:
# Competitions Input
competitions_df = api.competitions()
competitions_df[competitions_df["country_name"] == "International"]

,season_id,competition_id,competition_name,country_name,competition_gender,season_name
25,274,1470,FIFA U20 World Cup,International,male,1979
26,106,43,FIFA World Cup,International,male,2022
27,3,43,FIFA World Cup,International,male,2018
28,55,43,FIFA World Cup,International,male,1990
29,54,43,FIFA World Cup,International,male,1986
30,51,43,FIFA World Cup,International,male,1974
31,272,43,FIFA World Cup,International,male,1970
32,270,43,FIFA World Cup,International,male,1962
33,269,43,FIFA World Cup,International,male,1958
68,107,72,Women's World Cup,International,female,2023


In [11]:
# Games Input (Competition_id, Season_id)
games_world_cup_2018_df = api.games(43, 106)
games_world_cup_2018_df[games_world_cup_2018_df['game_id'] == 3857292] # Germany vs Costa Rica

,game_id,season_id,competition_id,competition_stage,game_day,game_date,home_team_id,away_team_id,home_score,away_score,venue,referee
43,3857292,106,43,Group Stage,3,2022-12-01 21:00:00,795,770,2,4,Al Bayt Stadium,Stéphanie Frappart


In [12]:
# Teams Input (game_id)
teams_world_cup_2018_df = api.teams(3857292)
teams_world_cup_2018_df.head()

,team_id,team_name
0,770,Germany
1,795,Costa Rica


In [13]:
# Player Input (game_id)
players_df = api.players(3857292)
players_df

,game_id,team_id,player_id,player_name,nickname,jersey_number,is_starter,starting_position_id,starting_position_name,minutes_played
0,3857292,770,3053,Leroy Sané,None,19,True,17,Right Wing,101
1,3857292,770,3167,Antonio Rüdiger,None,2,True,5,Left Center Back,101
2,3857292,770,5562,Thomas Müller,None,13,True,23,Center Forward,66
3,3857292,770,5570,Manuel Neuer,None,1,True,1,Goalkeeper,101
4,3857292,770,5579,Joshua Kimmich,None,6,True,2,Right Back,101
5,3857292,770,6322,Niklas Süle,None,15,True,3,Right Center Back,94
6,3857292,770,6324,Leon Goretzka,None,8,True,9,Right Defensive Midfield,45
7,3857292,770,8227,Mario Götze,None,11,False,0,Substitute,35
8,3857292,770,8400,Serge Gnabry,None,10,True,21,Left Wing,101
9,3857292,770,8507,Lukas Klostermann,None,16,False,0,Substitute,56


In [19]:
# # Load 360 data
# list_competitions_ids = []
# list_game_ids = []

# competitions_df = api.competitions()
# for _, row in competitions_df.iterrows():
#     if (row['competition_gender'] == 'male'):
#         list_competitions_ids.append((row['competition_id'], row['season_id']))

# for competition_id, season_id in list_competitions_ids:
#     games_df = api.games(competition_id, season_id)
#     for _, row in games_df.iterrows():
#         list_game_ids.append((row['game_id'], row['home_team_id'], row['away_team_id']))

# for game_id, home_team_id, away_team_id in list_game_ids:
#     try:
#         this_game_events_df = api.events(game_id, load_360=True)
#         this_game_events_df = spadl.add_names(convert_to_actions(this_game_events_df, home_team_id))
#         this_game_events_df.to_csv(f'data/statsbomb_event_360_data/{game_id}_{home_team_id}_{away_team_id}_statsbomb_event_360.csv')
#     except FileNotFoundError:
#         print(f'File not found {game_id}-{home_team_id}-{away_team_id}')

game_id_test_360 = 3857292
home_team_id = 795
this_game_events_df = api.events(game_id_test_360, load_360=True)
this_game_events_df_spadl = spadl.add_names(convert_to_actions(this_game_events_df, home_team_id))
this_game_events_df_spadl_with_360 = pd.merge(this_game_events_df_spadl, this_game_events_df[["event_id", "visible_area_360", "freeze_frame_360"]], how="inner", left_on="original_event_id", right_on="event_id")
this_game_events_df_spadl_with_360
test = this_game_events_df_spadl_with_360[this_game_events_df_spadl_with_360["original_event_id"] == "89222d44-fc73-41a2-942f-dc837959873f"]["freeze_frame_360"].iloc[0]
test

[{'teammate': True,
  'actor': False,
  'keeper': False,
  'location': [35.703343423842526, 47.82158335683629]},
 {'teammate': True,
  'actor': False,
  'keeper': False,
  'location': [39.86484785911665, 37.47920635502216]},
 {'teammate': True,
  'actor': False,
  'keeper': False,
  'location': [39.9701023741769, 53.65421622286938]},
 {'teammate': True,
  'actor': False,
  'keeper': False,
  'location': [49.73576749702802, 20.97891242298732]},
 {'teammate': True,
  'actor': False,
  'keeper': False,
  'location': [59.420109407196556, 27.983413362881848]},
 {'teammate': True,
  'actor': False,
  'keeper': False,
  'location': [59.66363761585325, 21.090140214711028]},
 {'teammate': True,
  'actor': False,
  'keeper': False,
  'location': [59.816773623217664, 49.899903458594615]},
 {'teammate': True,
  'actor': False,
  'keeper': False,
  'location': [60.575602412050074, 6.942586464687409]},
 {'teammate': True,
  'actor': False,
  'keeper': False,
  'location': [60.92214655448752, 55.0283